# LSTM post-processing RF

## Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

## Read input

In [2]:
import os
os.chdir('/mnt/guanabana/raid/home/slomp006')

# Reference data (inputs containing band sequence (of 92), targets containing respective lcf sequence (of 4))
dense_train_inputs_df = pd.read_csv('Input/Dense/dense_train_input.csv')
dense_train_targets_df = pd.read_csv('Input/Dense/dense_train_targets.csv')
dense_vali_inputs_df = pd.read_csv('Input/Dense/dense_vali_input.csv')
dense_vali_targets_df = pd.read_csv('Input/Dense/dense_vali_targets.csv')

# Predictions
dense_RF_pred = pd.read_csv('Output/RF/DenseRFPredict.csv')
dense_LSTM_pred = pd.read_csv('Output/LSTM/LSTM_dense.csv')

# Dense RF pred does not contain a correct amount of rows (one or more samples does not have the proper length = 92)
grouped = dense_RF_pred.groupby('location_id')
dense_RF_pred = grouped.filter(lambda x: len(x) == 92)

# LSTM pred does not have a date column, make one first
dense_LSTM_pred['date'] = dense_LSTM_pred.groupby('location_id').cumcount() + 1

# Now also create validation set and use similar IDs
common_ids = set(dense_vali_inputs_df['location_id']).intersection(dense_RF_pred['location_id'])
common_ids_LSTM = set(dense_vali_inputs_df['location_id']).intersection(dense_LSTM_pred['location_id'])

dense_RF_pred = dense_RF_pred[dense_RF_pred['location_id'].isin(common_ids)]
dense_LSTM_pred = dense_LSTM_pred[dense_LSTM_pred['location_id'].isin(common_ids_LSTM)]

dense_vali_inputs_LSTM_df = dense_vali_inputs_df[dense_vali_inputs_df['location_id'].isin(common_ids_LSTM)]
dense_vali_targets_LSTM_df = dense_vali_targets_df[dense_vali_targets_df['location_id'].isin(common_ids_LSTM)]
dense_vali_inputs_df = dense_vali_inputs_df[dense_vali_inputs_df['location_id'].isin(common_ids)]
dense_vali_targets_df = dense_vali_targets_df[dense_vali_targets_df['location_id'].isin(common_ids)]
                                                                          
# Order by ID, then by date / year 
dense_RF_pred = dense_RF_pred.sort_values(by=['location_id', 'date'])
dense_vali_inputs_df = dense_vali_inputs_df.sort_values(by=['location_id', 'date'])
dense_vali_targets_df = dense_vali_targets_df.sort_values(by=['location_id', 'reference_year']) 

dense_LSTM_pred = dense_LSTM_pred.sort_values(by=['location_id', 'date'])
dense_vali_inputs_LSTM_df = dense_vali_inputs_LSTM_df.sort_values(by=['location_id', 'date'])
dense_vali_targets_LSTM_df = dense_vali_targets_LSTM_df.sort_values(by=['location_id', 'reference_year']) 

# When respectively divided by 4 and 92, aggregated and dense should have same length
print(len(common_ids), len(dense_RF_pred)/92, len(dense_vali_inputs_df)/92, len(dense_vali_targets_df)/4)
print(len(common_ids_LSTM), len(dense_LSTM_pred)/92, len(dense_vali_inputs_LSTM_df)/92, len(dense_vali_targets_LSTM_df)/4)

30438 30438.0 30438.0 30438.0
30674 30674.0 30674.0 30674.0


In [5]:
print("We now have 8 data frames, 2 for training:")
print("Training inputs:", dense_train_inputs_df.shape, "with targets:", dense_train_targets_df.shape, "\n")
      
print("2 for validation for RF and 1 for RF prediction:")
print("Validation inputs:", dense_vali_inputs_df.shape, "with targets:", dense_vali_targets_df.shape)
print("and predictions:", dense_RF_pred.shape, "\n")

print("And 2 for validation for LSTM and 1 for LSTM prediction:")
print("Validation inputs:", dense_vali_inputs_LSTM_df.shape, "with targets:", dense_vali_targets_LSTM_df.shape)
print("and predictions:", dense_LSTM_pred.shape, "\n")

We now have 8 data frames, 2 for training:
Training inputs: (3085680, 13) with targets: (134160, 17) 

2 for validation for RF and 1 for RF prediction:
Validation inputs: (2800296, 12) with targets: (121752, 15)
and predictions: (2800296, 14) 

And 2 for validation for LSTM and 1 for LSTM prediction:
Validation inputs: (2822008, 12) with targets: (122696, 15)
and predictions: (2822008, 15) 



## Create tensors from input

In [6]:
# Classes are targets
targets = ['bare', 'crops',
       'grassland', 'shrub', 'tree', 'urban_built_up', 'water']

# Original input variables (now raw band values are input)
inputs = ['x', 'y', 'b1', 'b2', 'b3', 'b4',
         'b5', 'b6', 'b7']

# Also save IDs for later
IDs_aggregated = ['sample_id', 'location_id', 'reference_year', 'x', 'y']
IDs_dense = ['location_id', 'date', 'x', 'y']

# Create predictions list
RF_pred_list = []
LSTM_pred_list = []

# Validation list
vali_target_list = []
LSTM_vali_target_list = []

# Training
train_target_list = []

for colname in targets: 
    # Get column of feature
    col_RF = dense_RF_pred[colname]
    col_LSTM = dense_LSTM_pred[colname]
    col_training = dense_train_targets_df[colname]
    col_vali = dense_vali_targets_df[colname]
    col_vali_LSTM = dense_vali_targets_LSTM_df[colname]

    # Create tensors with sequence lengths of 92 for prediction, and 4 for training and vali (reference is aggregated)
    RF_tensor = torch.tensor(col_RF.values).view(-1, 92, 1).to(dtype=torch.float32)
    LSTM_tensor = torch.tensor(col_LSTM.values).view(-1, 92, 1).to(dtype=torch.float32)
    training_tensor = torch.tensor(col_training.values).view(-1, 4, 1).to(dtype=torch.float32)
    vali_tensor = torch.tensor(col_vali.values).view(-1, 4, 1).to(dtype=torch.float32)
    LSTM_vali_tensor = torch.tensor(col_vali_LSTM.values).view(-1, 4, 1).to(dtype=torch.float32)

    # Append to lists
    RF_pred_list.append(RF_tensor)
    LSTM_pred_list.append(LSTM_tensor)
    train_target_list.append(training_tensor)
    vali_target_list.append(vali_tensor)
    LSTM_vali_target_list.append(LSTM_vali_tensor)
    
# We now each class as a separate tensor but we want them as one. So concatenate the tensors along the last dimension
tensor_RF_preds = torch.cat(RF_pred_list, dim=-1)
tensor_LSTM_preds = torch.cat(LSTM_pred_list, dim=-1)
tensor_train_targets = torch.cat(train_target_list, dim=-1)
tensor_vali_targets = torch.cat(vali_target_list, dim=-1)
LSTM_tensor_vali_targets = torch.cat(LSTM_vali_target_list, dim=-1)

# Do the same for the input variables
train_input_list = []
vali_input_list = []
LSTM_vali_input_list = []

for colname in inputs:
    col_train = dense_train_inputs_df[colname]
    col_vali = dense_vali_inputs_df[colname]
    col_vali_LSTM = dense_vali_inputs_LSTM_df[colname]
    
    # Band sequence is sequence of 92
    train_tensor = torch.tensor(col_train.values).view(-1, 92, 1).to(dtype=torch.float32)
    vali_tensor = torch.tensor(col_vali.values).view(-1, 92, 1).to(dtype=torch.float32)
    LSTM_vali_tensor = torch.tensor(col_vali_LSTM.values).view(-1, 92, 1).to(dtype=torch.float32)
    
    train_input_list.append(train_tensor)
    vali_input_list.append(vali_tensor)
    LSTM_vali_input_list.append(LSTM_vali_tensor)

tensor_train_inputs = torch.cat(train_input_list, dim=-1)
tensor_vali_inputs = torch.cat(vali_input_list, dim=-1)
LSTM_tensor_vali_inputs = torch.cat(LSTM_vali_input_list, dim=-1)

# And for IDs (if no NaN values, not necessary)
Agg_ID_list = []
Dense_ID_list = []
LSTM_Agg_ID_list = []
LSTM_Dense_ID_list = []

# IDs for aggregated output
for colname in IDs_aggregated:
    ID_col = dense_vali_targets_df[colname]
    LSTM_ID_col = dense_vali_targets_LSTM_df[colname]
    ID_tensor = torch.tensor(ID_col.values).view(-1, 4, 1).to(dtype=torch.float32)
    LSTM_ID_tensor = torch.tensor(LSTM_ID_col.values).view(-1, 4, 1).to(dtype=torch.float32)
    Agg_ID_list.append(ID_tensor)
    LSTM_Agg_ID_list.append(LSTM_ID_tensor)

# IDs for dense output
for colname in IDs_dense:
    if colname == "date":
        ID_col = dense_RF_pred[colname].str.replace("-", "").astype(int) # make 2015-01-07 to 20150107 int value
#         ID_col = dense_RF_pred[colname]
        LSTM_ID_col = dense_LSTM_pred[colname]
    else:
        ID_col = dense_RF_pred[colname]
        LSTM_ID_col = dense_LSTM_pred[colname]
        
    ID_tensor = torch.tensor(ID_col.values).view(-1, 92, 1).to(dtype=torch.float32)
    LSTM_ID_tensor = torch.tensor(LSTM_ID_col.values).view(-1, 92, 1).to(dtype=torch.float32)
    Dense_ID_list.append(ID_tensor)
    LSTM_Dense_ID_list.append(LSTM_ID_tensor)
       
tensor_Agg_IDs = torch.cat(Agg_ID_list, dim=-1)
tensor_Dense_IDs = torch.cat(Dense_ID_list, dim=-1)
LSTM_tensor_Agg_IDs = torch.cat(LSTM_Agg_ID_list, dim=-1)
LSTM_tensor_Dense_IDs = torch.cat(LSTM_Dense_ID_list, dim=-1)

# Check for NaN 
tensor_RF_preds_old = tensor_RF_preds
tensor_LSTM_preds_old = tensor_LSTM_preds
tensor_train_inputs_old = tensor_train_inputs
tensor_vali_inputs_old = tensor_vali_inputs
LSTM_tensor_vali_inputs_old = LSTM_tensor_vali_inputs

# create an empty mask
mask = None

# Remove for each tensor that need to be similar
if torch.isnan(tensor_RF_preds).any():
    mask = torch.isnan(tensor_RF_preds).any(dim=1).any(dim=1)
    tensor_RF_preds = tensor_RF_preds[~mask]
    tensor_vali_inputs = tensor_vali_inputs[~mask]
    tensor_vali_targets = tensor_vali_targets[~mask]
    tensor_Agg_IDs = tensor_Agg_IDs[~mask]
    tensor_Dense_IDs = tensor_Dense_IDs[~mask]

if torch.isnan(tensor_LSTM_preds).any():
    mask = torch.isnan(tensor_LSTM_preds).any(dim=1).any(dim=1)
    tensor_LSTM_preds = tensor_LSTM_preds[~mask]
    LSTM_tensor_vali_inputs = LSTM_tensor_vali_inputs[~mask]
    LSTM_tensor_vali_targets = LSTM_tensor_vali_targets[~mask]
    LSTM_tensor_Agg_IDs = LSTM_tensor_Agg_IDs[~mask]
    LSTM_tensor_Dense_IDs = LSTM_tensor_Dense_IDs[~mask]
    
if torch.isnan(tensor_vali_inputs).any():
    mask = torch.isnan(tensor_vali_inputs).any(dim=1).any(dim=1)
    tensor_RF_preds = tensor_RF_preds[~mask]
    tensor_vali_inputs = tensor_vali_inputs[~mask]
    tensor_vali_targets = tensor_vali_targets[~mask]
    tensor_Agg_IDs = tensor_Agg_IDs[~mask]
    tensor_Dense_IDs = tensor_Dense_IDs[~mask]
    
if torch.isnan(tensor_train_inputs).any():
    mask = torch.isnan(tensor_train_inputs).any(dim=1).any(dim=1)
    tensor_train_inputs = tensor_train_inputs[~mask]
    tensor_train_targets = tensor_train_targets[~mask]
       
# Check final tensors (including shape)
print("RF dense predictions contained NaN values:", torch.isnan(tensor_RF_preds_old).any())
print("Validation band sequence contained NaN values:", torch.isnan(tensor_vali_inputs_old).any(), "\n")
print("RF dense predictions has shape:", tensor_RF_preds.shape)
print("Vali band input sequence has shape:", tensor_vali_inputs.shape)
print("Vali lcf sequence has shape:", tensor_vali_targets.shape)
print("Corresponding dense IDs has shape", tensor_Dense_IDs.shape, "and aggregated IDs:", tensor_Agg_IDs.shape, "\n")

print("LSTM dense predictions contained NaN values:", torch.isnan(tensor_LSTM_preds_old).any())
print("Validation band sequence contained NaN values:", torch.isnan(LSTM_tensor_vali_inputs_old).any(), "\n")
print("RF dense predictions has shape:", tensor_LSTM_preds.shape)
print("Vali band input sequence has shape:", LSTM_tensor_vali_inputs.shape)
print("Vali lcf sequence has shape:", LSTM_tensor_vali_targets.shape)
print("Corresponding dense IDs has shape", LSTM_tensor_Dense_IDs.shape, "and aggregated IDs:", LSTM_tensor_Agg_IDs.shape, "\n")

print("Train band sequence contained NaN values:", torch.isnan(tensor_train_inputs_old).any(), "\n")
print("Train input sequence has shape:", tensor_train_inputs.shape)
print("Train lcf sequence has shape:", tensor_train_targets.shape)


RF dense predictions contained NaN values: tensor(True)
Validation band sequence contained NaN values: tensor(False) 

RF dense predictions has shape: torch.Size([28507, 92, 7])
Vali band input sequence has shape: torch.Size([28507, 92, 9])
Vali lcf sequence has shape: torch.Size([28507, 4, 7])
Corresponding dense IDs has shape torch.Size([28507, 92, 4]) and aggregated IDs: torch.Size([28507, 4, 5]) 

LSTM dense predictions contained NaN values: tensor(False)
Validation band sequence contained NaN values: tensor(False) 

RF dense predictions has shape: torch.Size([30674, 92, 7])
Vali band input sequence has shape: torch.Size([30674, 92, 9])
Vali lcf sequence has shape: torch.Size([30674, 4, 7])
Corresponding dense IDs has shape torch.Size([30674, 92, 4]) and aggregated IDs: torch.Size([30674, 4, 5]) 

Train band sequence contained NaN values: tensor(False) 

Train input sequence has shape: torch.Size([33540, 92, 9])
Train lcf sequence has shape: torch.Size([33540, 4, 7])


## Create synthetic training data

#### Upsample and normalise

In [7]:
# First upsample some urban cases as was done before
# Function to upsample classes where value is > 0 (thus every sample where class is represented)
def upsample(X, Y, i, n):
    idx = Y[:, -1, i] != 0 # -1 to account for one that is already in the data set
    X_sub = X[idx].repeat(n-1, 1, 1)
    Y_sub = Y[idx].repeat(n-1, 1, 1)
    return X_sub, Y_sub

# Function to only upsample classes where value = 100 (thus every sample where class is 100)
def upsample100(X, Y, i, n):
    idx = Y[:, -1, i] == 100
    X_sub = X[idx].repeat(n-1, 1, 1)
    Y_sub = Y[idx].repeat(n-1, 1, 1)
    return X_sub, Y_sub

urban_X, urban_Y = upsample(tensor_train_inputs, tensor_train_targets, 5, 5) # repeat indice 5 (urban) times 5 where it is represented
urban_100_X, urban_100_Y = upsample100(tensor_train_inputs, tensor_train_targets, 5, 100) # repeat indice 5 (urban) times 100 where it is represented with 100

# Create upsampled training data (and keep Y information)
X_train = torch.cat([tensor_train_inputs, urban_X, urban_100_X], dim=0)
Y_train = torch.cat([tensor_train_targets, urban_Y, urban_100_Y], dim=0)
Y_coord = X_train[:, :, 1].unsqueeze(2) # Y coordinate is indice 1 (X = 0)

# Overwrite with normal training data (and keep Y information)
X_train = torch.cat([tensor_train_inputs], dim=0)
Y_train = torch.cat([tensor_train_targets], dim=0)
Y_coord = X_train[:, :, 1].unsqueeze(2) # Y coordinate is indice 1 (X = 0)

# Normalise
X_mean = X_train.mean(dim=0)
X_std = X_train.std(dim=0)

# Standardize the training set
X_train = (X_train - X_mean) / X_std

# Add unnormalized Y coordinate to X tensor
X_train = torch.cat([Y_coord, X_train], dim=-1)

# Create the TensorDataset for the training set
train_dataset = TensorDataset(X_train, Y_train)

print(X_train.shape, Y_train.shape)

torch.Size([33540, 92, 10]) torch.Size([33540, 4, 7])


#### Simple LSTM model to generate synthetic noise data

In [8]:
import torch.optim as optim
import torch.nn.init as init

# Model
class SynthModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0):
        super().__init__()
        # LSTM 
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Activation
        self.activation = nn.Tanh()
        
        # Linear with Xavier Uniform
        self.linear = nn.Linear(hidden_size, output_size)
        init.xavier_uniform_(self.linear.weight)
        
        # Softmax
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x):
        # Put input through LSTM layer
        x, _ = self.lstm(x)
        
        # Put LSTM output through dropout layer
        x = self.dropout(x)
        
        # Apply tanh activation function
        x = self.activation(x)
        
        # Linear transform x to shape [batch size, sequence length = 92, output size = 7]
        x = self.linear(x)
        
        # Make sure output distribution sums to 100
        x = self.softmax(x) * 100
       
        return x

In [9]:
# Input size is number of input variables and output size is number of classes
synthmodel = SynthModel(input_size=len(inputs), hidden_size=128, output_size=len(targets)) 

# Define the loss function and optimizer
loss_fn = nn.L1Loss()
optimizer = torch.optim.Adam(synthmodel.parameters(), lr=0.01, weight_decay=0.0)

# Create a DataLoader for the training set and validation sets (specify batch size)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1028, shuffle=True) 

# Retain data of best loss
best_loss = float("inf")
best_epoch = 0

# Epochs
num_epochs = 10 # The higher, the less noise

for epoch in range(num_epochs):
    
    # Progress
    print("\rAt epoch: {}/{}, Best loss: {} (obtained at epoch {})".format(epoch, 
                                                                            num_epochs, 
                                                                            round(best_loss, 3),
                                                                            best_epoch), end='\r')
    
    # Store epoch results in list
    epoch_pred = []
    epoch_pred_tensor = []
    epoch_actual = []
    epoch_actual_tensor = []
    epoch_loss = []
    
    # Retain input tensor for later
    input_tensor = []
    
    # Loop over the training set
    for X, Y in train_loader:

        input_tensor.append(X)
        
        # Clear the gradients
        optimizer.zero_grad()

        # Forward pass
        Y_pred = synthmodel(X[:, :, 1:]) # Don't use unnormalised Y coord
        Y = Y.repeat_interleave(repeats=23, dim=1)

        # Compute the loss
        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss)

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()
        
        # Predictions per batch
        Y_pred_list = [x.tolist() for x in Y_pred]
        Y_list = [x.tolist() for x in Y]
        
        # Add batch prediction to list
        epoch_pred.append(Y_pred_list)
        epoch_actual.append(Y_list)
        epoch_pred_tensor.append(Y_pred)
        epoch_actual_tensor.append(Y)
        
    # Retain a tensor format for the Markov Chain    
    epoch_pred_tensor = torch.cat(epoch_pred_tensor, dim=0)
    epoch_actual_tensor = torch.cat(epoch_actual_tensor, dim=0)
    
    # Also retain input tensor
    input_tensor = torch.cat(input_tensor, dim=0)
    
    avg_loss = sum(epoch_loss)/len(epoch_loss)
    
    # Retain prediction if best loss
    if avg_loss < best_loss:
        best_loss = avg_loss.item()
        best_pred = epoch_pred
        best_actual = epoch_actual
        best_epoch = epoch
        best_pred_tensor = epoch_pred_tensor
        best_actual_tensor = epoch_actual_tensor
        best_input_tensor = input_tensor[:, :, 1:]
        best_Y_coord = input_tensor[:, :, 0].unsqueeze(2)
        
print("\n", "Done.", "Result at epoch", best_epoch, "is taken.")

At epoch: 4/5, Best loss: 13.534 (obtained at epoch 3)
 Done. Result at epoch 4 is taken.


#### Save tensors to file for later

In [10]:
# torch.save(best_pred_tensor.detach(), 'LSTM(RF)/Synth/pred_tensor.pt')
# torch.save(best_actual_tensor.detach(), 'LSTM(RF)/Synth/actual_tensor.pt')
# torch.save(best_input_tensor.detach(), 'LSTM(RF)/Synth/input_tensor.pt')
# torch.save(best_Y_coord.detach(), 'LSTM(RF)/Synth/Y_coord_tensor.pt')

# # Use torch.load(path) to retrieve them 

#### Check synthesized (noise) data

In [9]:
import math
from statistics import mean
from sklearn.metrics import mean_squared_error

# # Make lists of predictions and actual fractions (reference)
nested_pred = best_pred # results from the epoch with the minimum loss is taken
nested_actual = best_actual

unnested_pred = []
for data in nested_pred:
  for batch in data:
    for timestep in batch:
      for target in range(len(timestep)):
        unnested_pred.append(timestep[target])

unnested_actual = []
for data in nested_actual:
  for batch in data:
    for timestep in batch:
      for target in range(len(timestep)):
        unnested_actual.append(timestep[target])

# These lists contain output for entire predictions 
# Now retain lists for each class
pred_class = {}
true_class = {}

# Initialize lists for each class in predictions
for i in range(len(targets)):
  pred_class[f'{targets[i]}'] = unnested_pred[i::len(targets)]

# Initialize lists for each class in reference data
for i in range(len(targets)):
  true_class[f'{targets[i]}'] = unnested_actual[i::len(targets)]

RMSEavg = 0
MAEavg = 0

# Plot the lists as graphs

# Loop through the data and plot the actual and predicted values
for i in range(len(targets)):
    # Get the data for the current class
    true = true_class[f'{targets[i]}']
    predicted = pred_class[f'{targets[i]}']

    # Define the x-axis data as a range of values from 0 to the length of the data
    x = range(len(true))

    # Create a new figure
    fig = plt.figure(i)

    # Create a figure with certain size
    fig = plt.figure(figsize=(20, 3))

    # Create axes
    ax = fig.add_subplot(1, 1, 1)

    # Plot the actual and predicted values
    ax.plot(x, true, label='Actual')
    ax.plot(x, predicted, label='Synthesized')

    # Add a legend
    ax.legend()

    # Set the title using the class name
    var_name = f'{targets[i]}'
    ax.set_title(var_name)

    # Show the figure
    plt.show()

    # Print RMSE / MAE
    rmse = mean_squared_error(predicted, true) ** 0.5
    RMSEavg = RMSEavg + rmse
    print(f'RMSE for {var_name}: {round(rmse, 2)}')

    difference = [abs(predicted - true) for predicted, true in zip(predicted, true)]
    mae = mean(difference)
    MAEavg = MAEavg + mae
    print(f'MAE for {var_name}: {round(mae, 2)}')

print("\n")
RMSEavg = RMSEavg / len(targets)
MAEavg = MAEavg / len(targets)

print(f'Average RMSE is {round(RMSEavg, 2)} and average MAE is {round(MAEavg, 2)}')

#### Create training data

In [12]:
# Tensors
synth_lcf = best_pred_tensor.detach()
actual_lcf = best_actual_tensor.detach()
input_vars = best_input_tensor.detach()
Y_coord_training = best_Y_coord.detach()

# Training X (inputted into the model)
inputs_plus_actual = torch.cat([Y_coord_training, input_vars, actual_lcf], dim=-1)
inputs_plus_synth = torch.cat([Y_coord_training, input_vars, synth_lcf], dim=-1)
training_X = torch.cat([inputs_plus_actual, inputs_plus_synth], dim=0)

print("We have three train X: using actual lcf:", inputs_plus_actual.shape, "using synth data:", inputs_plus_synth.shape)
print("and both:", training_X.shape, "\n")

# Training X only inputs are normalised, also make one with normalised lcf
inputs_plus_actual_norm = torch.cat([input_vars, actual_lcf], dim=-1)
inputs_plus_synth_norm = torch.cat([input_vars, synth_lcf], dim=-1)

X_mean_all = training_X[:, :, 1:].mean(dim=0)
X_std_all = training_X[:, :, 1:].std(dim=0)

inputs_plus_actual_norm = (inputs_plus_actual_norm - X_mean_all) / X_std_all
inputs_plus_synth_norm = (inputs_plus_synth_norm - X_mean_all) / X_std_all

# Now add unnormalised Y coordinate again
inputs_plus_actual_norm = torch.cat([Y_coord_training, inputs_plus_actual_norm], dim=-1)
inputs_plus_synth_norm = torch.cat([Y_coord_training, inputs_plus_synth_norm], dim=-1)
training_X_norm = torch.cat([inputs_plus_actual_norm, inputs_plus_synth_norm], dim=0)

print("x2, as we have a normalised one and a non normalised one", "\n")

# Training Y (what the model should be trained to predict)
actual_lcf1 = actual_lcf[:, 0:22, :].mean(dim=1).unsqueeze(1)
actual_lcf2 = actual_lcf[:, 23:55, :].mean(dim=1).unsqueeze(1)
actual_lcf3 = actual_lcf[:, 56:78, :].mean(dim=1).unsqueeze(1)
actual_lcf4 = actual_lcf[:, 79:91, :].mean(dim=1).unsqueeze(1)
actual_lcf_agg = torch.cat([actual_lcf1, actual_lcf2, actual_lcf3, actual_lcf4], dim=1)
training_Y = torch.cat([actual_lcf_agg, actual_lcf_agg], dim=0)

print("We have three respective train Y:", actual_lcf_agg.shape, actual_lcf_agg.shape, "and", training_Y.shape)


We have three train X: using actual lcf: torch.Size([33540, 92, 17]) using synth data: torch.Size([33540, 92, 17])
and both: torch.Size([67080, 92, 17]) 

x2, as we have a normalised one and a non normalised one 

We have three respective train Y: torch.Size([33540, 4, 7]) torch.Size([33540, 4, 7]) and torch.Size([67080, 4, 7])


#### Regress samples
Instead of repeating, perhaps regressed samples are better

In [13]:
def RegressSamples(originalX, originalY):    
    
    # Clone original tensors
    test_Y = originalY.clone().detach()
    test_X = originalX.clone().detach()

    # Save regressed tensor
    regressed_tensor = []

    # Loop over samples
    for sample in range(test_Y.shape[0]):

        # Progress
        print("\rRegressing sample: {}/{}".format(sample, test_Y.shape[0]), end='\r')

        # Create output tensor sample for sample      
        out = torch.zeros(1, 92, 7)

        # If southern hemisphere, place fractions at first months (important, first indice need to translate to Y coord!!)      
        if (test_X[sample, :, 0] < 0).all():

            out[:, 0:5, :] = test_Y[sample, 0, :].unsqueeze(0).unsqueeze(1).repeat_interleave(5, dim=1)
            out[:, 23:28, :] = test_Y[sample, 1, :].unsqueeze(0).unsqueeze(1).repeat_interleave(5, dim=1)
            out[:, 46:51, :] = test_Y[sample, 2, :].unsqueeze(0).unsqueeze(1).repeat_interleave(5, dim=1)
            out[:, 69:, :] = test_Y[sample, 3, :].unsqueeze(0).unsqueeze(1).repeat_interleave(23, dim=1)

            diff1 = (out[:, 23, :] - out[:, 4, :]).unsqueeze(1)
            diff2 = (out[:, 46, :] - out[:, 27, :]).unsqueeze(1)
            diff3 = (out[:, 69, :] - out[:, 50, :]).unsqueeze(1)

            for diff in range(3):

                if diff == 0:
                    diff_timesteps = 23-5
                    start_timestep = 5-1
                    diff_values = diff1
                elif diff == 1:
                    diff_timesteps = 46-28
                    start_timestep = 28-1
                    diff_values = diff2
                else:
                    diff_timesteps = 69-51
                    start_timestep = 51-1
                    diff_values = diff3

                regr_values = []

                for i in range(diff_values.shape[2]):

                    if diff_values[:, :, i] == 0:
                        regr_value = torch.zeros(1, 1).unsqueeze(1)
                    else:
                        regr_value = torch.tensor(diff_values[:, :, i] / diff_timesteps).unsqueeze(1)

                    regr_values.append(regr_value)

                regr_values = torch.cat(regr_values, dim=-1)

                for next_step in range(diff_timesteps):
                    timestep = out[:, start_timestep+next_step, :].unsqueeze(1)
                    out[:, start_timestep+next_step+1, :] = timestep + regr_values

        # If northern hemisphere, place fractions at the middle months      
        else:

            out[:, 0:14, :] = test_Y[sample, 0, :].unsqueeze(0).unsqueeze(1).repeat_interleave(14, dim=1)
            out[:, 32:37, :] = test_Y[sample, 1, :].unsqueeze(0).unsqueeze(1).repeat_interleave(5, dim=1)
            out[:, 55:60, :] = test_Y[sample, 2, :].unsqueeze(0).unsqueeze(1).repeat_interleave(5, dim=1)
            out[:, 78:, :] = test_Y[sample, 3, :].unsqueeze(0).unsqueeze(1).repeat_interleave(14, dim=1)

            diff1 = (out[:, 32, :] - out[:, 13, :]).unsqueeze(1)
            diff2 = (out[:, 55, :] - out[:, 36, :]).unsqueeze(1)
            diff3 = (out[:, 78, :] - out[:, 59, :]).unsqueeze(1)

            for diff in range(3):

                if diff == 0:
                    diff_timesteps = 32-14
                    start_timestep = 14-1
                    diff_values = diff1
                elif diff == 1:
                    diff_timesteps = 55-37
                    start_timestep = 37-1
                    diff_values = diff2
                else:
                    diff_timesteps = 78-60
                    start_timestep = 60-1
                    diff_values = diff3

                regr_values = []

                for i in range(diff_values.shape[2]):

                    if diff_values[:, :, i] == 0:
                        regr_value = torch.zeros(1, 1).unsqueeze(1)
                    else:
                        regr_value = torch.tensor(diff_values[:, :, i] / diff_timesteps).unsqueeze(1)

                    regr_values.append(regr_value)

                regr_values = torch.cat(regr_values, dim=-1)

                for next_step in range(diff_timesteps):
                    timestep = out[:, start_timestep+next_step, :].unsqueeze(1)
                    out[:, start_timestep+next_step+1, :] = timestep + regr_values

        # Append regressed sample to full regressed tensor list      
        regressed_tensor.append(out)

    # Concatenate into full shape [num samples, 92, 7]          
    regressed_tensor = torch.cat(regressed_tensor, dim=0)

    # Return tensor
    return regressed_tensor

# Perform function on data
# Important, indice 0 in argument 1 needs to translate to unnormalised Y coord
training_Y_regressed = RegressSamples(training_X, training_Y)
training_Y_regressed_actual = training_Y_regressed[:int(training_Y_regressed.shape[0]/2), :, :]
training_Y_regressed_synth = training_Y_regressed[int(training_Y_regressed.shape[0]/2):, :, :]

print("\n", "We now have regressed", training_Y.shape, "into", training_Y_regressed.shape)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:97: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Regressing sample: 67079/67080
 We now have regressed torch.Size([67080, 4, 7]) into torch.Size([67080, 92, 7])


#### Check if regressed properly

In [14]:
# Check certain sample
i = 4

torch.set_printoptions(sci_mode=False, precision=2, linewidth=100)

print("Original sample:")
print(training_Y[i, :, :].detach(), "\n")
print("Regressed sample:")
print(training_Y_regressed[i, :, :].detach())

Original sample:
tensor([[ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.]]) 

Regressed sample:
tensor([[ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.,  0., 88., 12.,  0.,  0.],
        [ 0.,  0.

#### Oversample change samples 

In [15]:
def OversampleChange(originalX, originalY, repeattimes):
    
    # Create a tensor with all samples that have experienced change
    # Make sure X and Y correspond with each other
    changed_tensorsY_list = []
    changed_tensorsX_list = []

    # Loop through samples, break if only one timestep has change
    for i in range(originalY.size(0)): 
        for j in range(originalY.size(1)-1):  
            if torch.any(originalY[i, j, :] != originalY[i, j+1, :]):

                # Append Y change tensors
                change_tensor = originalY[i, :, :].unsqueeze(0)
                changed_tensorsY_list.append(change_tensor)

                # Append X change tensors
                change_tensor_input = originalX[i, :, :].unsqueeze(0)
                changed_tensorsX_list.append(change_tensor_input)

                break

    # Create change tensors            
    changed_tensors = torch.cat(changed_tensorsY_list, dim=0)
    changed_tensors_input = torch.cat(changed_tensorsX_list, dim=0)
    
    # Repeat change tensors x amount of times
    changed_tensors_repeated = changed_tensors.detach().repeat(repeattimes, 1, 1) 
    changed_tensors_input_repeated = changed_tensors_input.detach().repeat(repeattimes, 1, 1)
    
    # Concatenate X and Y back
    originalX_oversampled = torch.cat([originalX, changed_tensors_input_repeated], dim=0)
    originalY_oversampled = torch.cat([originalY, changed_tensors_repeated], dim=0)
    
    # Print changed size
    print("Done oversampling change. Changed size", originalX.size(0), "into", originalX_oversampled.size(0), "\n",
         "(", changed_tensors.size(0), "change samples into", changed_tensors_repeated.size(0), ")", "\n")
    
    # Return oversampled X and Y
    return originalX_oversampled, originalY_oversampled
    
# Execute function on data (normalised LCF and non normalised LCF)
repeat_number = 10 # Repeating 10 times will give a ~1:1 ratio of change/non-change

training_X_oversampled, training_Y_oversampled = OversampleChange(training_X, training_Y, repeat_number)
training_X_norm_oversampled, _ = OversampleChange(training_X_norm, training_Y, repeat_number) # Y oversampled discarded as it is the same 
_, training_Y_regressed_oversampled = OversampleChange(training_X, training_Y_regressed, repeat_number) # X oversampled discarded as its the same

# oversampling a regressed sample might take longer (loop over 92 sequence instead of 4)

Done oversampling change. Changed size 67080 into 132400 
 ( 6532 change samples into 65320 ) 

Done oversampling change. Changed size 67080 into 132400 
 ( 6532 change samples into 65320 ) 

Done oversampling change. Changed size 67080 into 132400 
 ( 6532 change samples into 65320 ) 



#### Create validation data

In [16]:
# The way input tensors are set up:
tensor_RF_all = torch.cat([tensor_vali_inputs, tensor_RF_preds], dim=-1)
tensor_LSTM_all = torch.cat([LSTM_tensor_vali_inputs, tensor_LSTM_preds], dim=-1)
# This means that the last 7 columns represent the land cover classes

# Split into inputs and land cover fractions
RF_X_inputs = tensor_RF_all[:, :, :-len(targets)]
RF_X_lcf = tensor_RF_all[:, :, -len(targets):]
LSTM_X_inputs = tensor_LSTM_all[:, :, :-len(targets)]
LSTM_X_lcf = tensor_LSTM_all[:, :, -len(targets):]

# Normalise inputs 
X_mean_inputs = tensor_train_inputs.mean(dim=0)
X_std_inputs = tensor_train_inputs.std(dim=0)
RF_X_inputs_normalised = (RF_X_inputs - X_mean_inputs) / X_std_inputs
LSTM_X_inputs_normalised = (LSTM_X_inputs - X_mean_inputs) / X_std_inputs

# Rejoin data (land cover fractions are thus not normalised)
RF_X = torch.cat([RF_X_inputs[:, :, 1].unsqueeze(2), RF_X_inputs_normalised, RF_X_lcf], dim=-1) # indice 1 here represents Y coord
LSTM_X = torch.cat([LSTM_X_inputs[:, :, 1].unsqueeze(2), LSTM_X_inputs_normalised, LSTM_X_lcf], dim=-1)

# Now do the same, but also normalise land cover fractions (can use x mean and std defined before)
RF_X_norm = (RF_X[:, :, 1:] - X_mean_all) / X_std_all
RF_X_norm = torch.cat([RF_X_inputs[:, :, 1].unsqueeze(2), RF_X_norm], dim=-1)
LSTM_X_norm = (LSTM_X[:, :, 1:] - X_mean_all) / X_std_all
LSTM_X_norm = torch.cat([LSTM_X_inputs[:, :, 1].unsqueeze(2), LSTM_X_norm], dim=-1)

# And the to be predicted Y
RF_Y = tensor_vali_targets
LSTM_Y = LSTM_tensor_vali_targets

# Print validation data
print("We now have (an unnormalised and normalised) validation RF X:", RF_X.shape, RF_X_norm.shape)
print("And a to be predicted validation RF Y:", RF_Y.shape, "\n")
print("And (an unnormalised and normalised) validation LSTM X:", LSTM_X.shape, LSTM_X_norm.shape)
print("And a to be predicted validation LSTM Y:", LSTM_Y.shape, "\n")


We now have (an unnormalised and normalised) validation RF X: torch.Size([28507, 92, 17]) torch.Size([28507, 92, 17])
And a to be predicted validation RF Y: torch.Size([28507, 4, 7]) 

And (an unnormalised and normalised) validation LSTM X: torch.Size([30674, 92, 17]) torch.Size([30674, 92, 17])
And a to be predicted validation LSTM Y: torch.Size([30674, 4, 7]) 



#### Create data sets

In [17]:
## We now have the following data sets

# Input data (Landsat sequence) is always normalised. 'Norm' refers to also normalised input LCF.

## Training
# Train dataset: using actual+synth
train_dataset = TensorDataset(training_X, training_Y)
train_dataset_norm = TensorDataset(training_X_norm, training_Y)
train_dataset_oversampled = TensorDataset(training_X_oversampled, training_Y_oversampled)
train_dataset_norm_oversampled = TensorDataset(training_X_norm_oversampled, training_Y_oversampled)

# Regressed data sets
train_dataset_regressed = TensorDataset(training_X, training_Y_regressed)
train_dataset_norm_regressed = TensorDataset(training_X_norm, training_Y_regressed)
train_dataset_regressed_oversampled = TensorDataset(training_X_oversampled, training_Y_regressed_oversampled)
train_dataset_norm_regressed_oversampled = TensorDataset(training_X_norm_oversampled, training_Y_regressed_oversampled)

# Using only actual/synth (does not include oversampled)
train_dataset_actual = TensorDataset(inputs_plus_actual, actual_lcf_agg)
train_dataset_synth = TensorDataset(inputs_plus_synth, actual_lcf_agg)
train_dataset_norm_actual = TensorDataset(inputs_plus_actual_norm, actual_lcf_agg)
train_dataset_norm_synth = TensorDataset(inputs_plus_synth_norm, actual_lcf_agg)
train_dataset_regressed_actual = TensorDataset(inputs_plus_actual, training_Y_regressed_actual)
train_dataset_regressed_synth = TensorDataset(inputs_plus_synth, training_Y_regressed_actual)
train_dataset_regressed_norm_actual = TensorDataset(inputs_plus_actual_norm, training_Y_regressed_actual)
train_dataset_regressed_norm_synth = TensorDataset(inputs_plus_synth_norm, training_Y_regressed_synth)

# Synth x2 (norm regressed)
synth_2x = torch.cat([inputs_plus_synth_norm, inputs_plus_synth_norm], dim=0)
lcf_2x = torch.cat([training_Y_regressed_synth, training_Y_regressed_synth], dim=0)
train_dataset_regressed_norm_synth_2x = TensorDataset(synth_2x, lcf_2x)

## Validation
# And of course prediction/validation data set
RF_vali_dataset = TensorDataset(RF_X, RF_Y)
RF_vali_dataset_norm = TensorDataset(RF_X_norm, RF_Y)
LSTM_vali_dataset = TensorDataset(LSTM_X, LSTM_Y)
LSTM_vali_dataset_norm = TensorDataset(LSTM_X_norm, LSTM_Y)

print("Data sets ready!")

Data sets ready!


## LSTM post-process model

#### Define model

In [18]:
import torch.optim as optim
import torch.nn.init as init

# Bidirectional LSTM (nr 3, full sequence input)
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1, dropout=0.3):
        super().__init__()
        
        input_size = input_size - 1 # disregard Y coordinate
        self.output_size = output_size
        self.hidden_size = hidden_size
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)
        
        # Dropout
        self.dropout = nn.Dropout(dropout)
        
        # Activation
        self.activation = nn.Tanh()
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size*2, output_size)
        
        # Softmax
        self.softmax = nn.Softmax(dim=-1)
        
    def forward(self, x_input):
        
        # Don't use Y coordinate (first indice)
        x = x_input[:, :, 1:]
        
        # Put inputs through LSTM layer 
        x, _ = self.lstm(x) # [batch size, 4, hidden size]
        
        # Apply dropout for overfitting
        x = self.dropout(x)
        
        # Tanh activation (non-linearity and zero-centering)
        x = self.activation(x)
        
        # Go through final fully connected layer
        x = self.fc(x) # [batch size, 4, 7]
        
        # Make sure output distribution sums to 100
        x = self.softmax(x) * 100
        
        # Create two outputs: dense and aggregated
        # Aggregate based on Y > 0 (northern hemisphere) and Y < 0 (southern hemisphere)
        agg_batch = []
        for i in range(x_input.shape[0]):
            if (x_input[i, :, 0] > 0).all(): # indice 1 is Y coordinate
                mean1 = x[i, 9:13,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                mean2 = x[i, 32:36,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                mean3 = x[i, 55:59,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                mean4 = x[i, 78:82,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                agg_sample = torch.cat([mean1, mean2, mean3, mean4], dim=1) # [1, 4, 7]
                agg_batch.append(agg_sample)      
            else:
                mean1 = x[i, 0:4,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                mean2 = x[i, 23:27,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                mean3 = x[i, 46:50,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                mean4 = x[i, 69:73,:].unsqueeze(0).mean(dim=1).unsqueeze(1)
                agg_sample = torch.cat([mean1, mean2, mean3, mean4], dim=1) # [1, 4, 7]
                agg_batch.append(agg_sample)  
        
        x_agg = torch.cat(agg_batch, dim=0)
                
        return x, x_agg
    

#### Create separate loss function
Calculate loss on change samples, all samples, and non-change samples

In [20]:
def MAE_Change(prediction, actual):
    # Compute a tensor that indicates which elements have changed
    change_mask = torch.any(actual[:, :-1, :] != actual[:, 1:, :], dim=-1)
    sample_indices, _ = torch.where(change_mask)
    
    # Check if any change samples are present
    if sample_indices.numel() == 0:
        return torch.tensor(0.0)
    
    # Actual and predicted change
    actual_change = actual[sample_indices]
    predicted_change = prediction[sample_indices]

    # Calculate MAE of the changes
#     mae = torch.mean(torch.abs(predicted_change - actual_change))
    mae = nn.L1Loss()(predicted_change, actual_change)

    return mae

def MAE_NoChange(prediction, actual):
    # Compute a tensor that indicates which elements have changed
    change_mask = torch.any(actual[:, :-1, :] != actual[:, 1:, :], dim=-1)
    sample_indices, _ = torch.where(change_mask)
    
    # Check if any change samples are present
    if sample_indices.numel() == 0:
        return torch.tensor(0.0)
    
    # Actual and predicted 
    actual_nochange = actual[~sample_indices]
    predicted_nochange = prediction[~sample_indices]

    # Calculate MAE of the non-changes
    mae = nn.L1Loss()(predicted_nochange, actual_nochange)

    return mae

# Combined loss
class CombinedLoss(nn.Module):
    def __init__(self, change_weight):
        super().__init__()
        self.change_weight = change_weight
        self.l1_loss = nn.L1Loss()

    def forward(self, prediction, actual):
        # Loss of all samples
        l1_loss = self.l1_loss(prediction, actual)
        
        # Loss of samples with no change
        no_change_loss = MAE_NoChange(prediction, actual)

        # Loss of samples with change
        change_loss = MAE_Change(prediction, actual)
        
        # Check if change loss is more than 0. If not, full weight on all samples
        if torch.all(change_loss == 0):
            weight = 0.0
        else:
            weight = self.change_weight

        # Combine losses with specified weights
        combined_loss = (1 - weight) * l1_loss + weight * change_loss

        return combined_loss


#### Train/run model

In [ ]:
# Make lists to retain loss
losses_train_epochs = []
losses_test_epochs = []

# Retain prediction at lowest loss
best_loss = float("inf")
current_loss = float("inf")
best_epoch = 0
best_layer = 0

# Model tested:
modeltype = "LSTM"

# Number of stacked layers (just break operation otherwise)
for layer in range(2):
    
    # Stop iterating if layer did not find better loss
    if layer - best_layer > 1:
        break

    # At the start, initialise with either LSTM or RF
    if layer == 0:
        
        if modeltype == "LSTM":          
            # Use synth and or actual data as start data set
            X_dataset = train_dataset_norm_regressed
            Y_dataset = LSTM_vali_dataset_norm
            
            # Test split
            train_size = int(0.8 * training_X_norm.size(0)) # was 0.5
            test_size = training_X_norm.size(0) - train_size
            X_dataset, test_dataset = random_split(X_dataset, [train_size, test_size])
            
            # Input length to start with
            start_input_length = training_X_norm.size(-1)
        
        elif modeltype == "RF":
            # Use synth and or actual data as start data set
            X_dataset = train_dataset_norm_regressed
            Y_dataset = RF_vali_dataset_norm
            
            # Test split
            train_size = int(0.8 * training_X_norm.size(0)) # was 0.5
            test_size = training_X_norm.size(0) - train_size
            X_dataset, test_dataset = random_split(X_dataset, [train_size, test_size])
            
            # Input length to start with
            start_input_length = training_X_norm.size(-1)
            
        else:
            print("No valid model input.")
            break

        # Initialise loss function    
        change_weight = 0.3 # between 0-1, the higher, the more focused on change 
        loss_fn = CombinedLoss(change_weight=change_weight)
#         loss_fn = nn.L1Loss()
        
        # Initialise model and optimizer
        model = BiLSTMModel(input_size = start_input_length, hidden_size = 256, output_size = 7)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001) # was 0.0003

        # Create a DataLoader for the training set and validation sets (specify batch size)
        train_loader = torch.utils.data.DataLoader(X_dataset, batch_size=256, shuffle=True) # was 128
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=True)
        vali_loader = torch.utils.data.DataLoader(Y_dataset, batch_size=256, shuffle=False) 

        # Epochs at start
        num_epochs = 100
        
        # Set the model to training mode 
        model.zero_grad()
        optimizer.zero_grad()
    
    else:

        # At the next layers, use newly created data sets
        X_dataset = new_train_dataset
        Y_dataset = new_vali_dataset 
        
        # Test split
        train_size = int(0.8 * new_input_size) # was 0.5
        test_size = new_input_size - train_size
        X_dataset, test_dataset = random_split(X_dataset, [train_size, test_size])
        
        # Create a DataLoader for the training set and validation sets (specify batch size)
        train_loader = torch.utils.data.DataLoader(X_dataset, batch_size=256, shuffle=True) # was 128
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=True)
        vali_loader = torch.utils.data.DataLoader(Y_dataset, batch_size=256, shuffle=False) 
        
        # Epochs
        num_epochs = 200
        
        # Re-initialise model and remove gradients from previous layer
        model = BiLSTMModel(input_size = new_input_length, hidden_size = 256, output_size = 7) 
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
        model.zero_grad()
        optimizer.zero_grad()

    # Loop over the training data
    for epoch in range(num_epochs):

        # Progress
        print("\rAt epoch: {}/{}, Loss: {}, Best loss: {} (obtained at epoch {}, layer {})".format(epoch, 
                                                                                num_epochs, round(current_loss, 3), round(best_loss, 3),
                                                                                best_epoch, best_layer+1), end='\r')

        # Loss per epoch
        epoch_trainloss = []
        epoch_testloss = []
        
        # Predictions per epoch
        epoch_pred_agg = []
        epoch_pred_dense = []
        epoch_pred_tensor = []
        epoch_pred_dense_tensor = []
        epoch_actual = []
        epoch_actual_tensor = []
        epoch_actual_dense_tensor = []
        
        # Save training input
        train_input = []
        
        # Save train target
        train_target = []
        
        # Start model training
        model.train() 
        
        # Loop over the training set
        for X, Y in train_loader:
            
            # Check if Y is already a sequence of 92 (regressed), or repeat it into 92
            if Y.shape[1] < 91:
                Y_dense = Y.repeat_interleave(repeats=23, dim=1)
            else:
                Y_dense = Y

            # Clear the gradients
            optimizer.zero_grad()

            # Compute the loss
            Y_pred, Y_pred_agg = model(X)
            loss = loss_fn(Y_pred, Y_dense)

            # Backward pass
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Append the loss to the lists
            epoch_trainloss.append(loss.item())

            # Save input for next layer
            X_without_prev_lcf = X[:, :, :-7]
            train_input.append(torch.cat([X_without_prev_lcf, Y_pred], dim=-1))
            train_target.append(Y)
        
        # Save test and vali input
        test_input = []
        vali_input = []
        
        # Save test and vali target
        test_target = []
        vali_target = []
        
        # Deactivatite autograd engine when looping over validation set
        with torch.no_grad():

            # Set the model to evaluation mode
            model.eval()
            
             # Loop over the test data set
            for X, Y in test_loader:

                # Check if Y is already a sequence of 92 (regressed), or repeat it into 92
                if Y.shape[1] < 91:
                    Y_dense = Y.repeat_interleave(repeats=23, dim=1)
                else:
                    Y_dense = Y

                # Compute the loss
                Y_pred, Y_pred_agg = model(X)
                loss = loss_fn(Y_pred, Y_dense)

                # Append the loss to the list
                epoch_testloss.append(loss.item())

                # Save input for next layer
                X_without_prev_lcf = X[:, :, :-7]
                test_input.append(torch.cat([X_without_prev_lcf, Y_pred], dim=-1))
                test_target.append(Y)
        
            # Loop over the vali data set
            for X, Y in vali_loader:
                
                # Validation data was not regressed
                Y_dense = Y.repeat_interleave(repeats=23, dim=1)

                # Make prediction    
                Y_pred, Y_pred_agg = model(X)
                
                # Store predictions for batch
                Y_pred_dense_list = [x.tolist() for x in Y_pred]
                Y_pred_agg_list = [x.tolist() for x in Y_pred_agg]
                Y_list = [x.tolist() for x in Y]

                # Add batch prediction to list 
                epoch_pred_dense.append(Y_pred_dense_list)
                epoch_pred_agg.append(Y_pred_agg_list)
                epoch_actual.append(Y_list)

                # Retain tensors per batch
                epoch_pred_dense_tensor.append(Y_pred)
                epoch_actual_dense_tensor.append(Y_dense)
                epoch_pred_tensor.append(Y_pred_agg)
                epoch_actual_tensor.append(Y)

                # Save input for next layer
                X_without_prev_lcf = X[:, :, :-7]
                vali_input.append(torch.cat([X_without_prev_lcf, Y_pred], dim=-1))
                vali_target.append(Y)
        
        # Retain tensors for whole epoch for statistics
        epoch_pred_dense_tensor = torch.cat(epoch_pred_dense_tensor, dim=0)
        epoch_pred_tensor = torch.cat(epoch_pred_tensor, dim=0)
        epoch_actual_tensor = torch.cat(epoch_actual_tensor, dim=0)
        epoch_actual_dense_tensor = torch.cat(epoch_actual_dense_tensor, dim=0)
    
        # Add epoch losses to total loss list 
        losses_train_epochs.append(epoch_trainloss)
        losses_test_epochs.append(epoch_testloss)

        # Retain new inputs/targets
        # Training
        train_input = torch.cat(train_input, dim=0)
        train_target = torch.cat(train_target, dim=0)
        
        # Test
        test_input = torch.cat(test_input, dim=0)
        test_target = torch.cat(test_target, dim=0)
        
        # Vali
        vali_input = torch.cat(vali_input, dim=0)
        vali_target = torch.cat(vali_target, dim=0)
        
        # Current loss
        current_loss = sum(epoch_testloss) / len(test_loader)
        
        if epoch == 0 and (layer == 1 or layer == 2 or layer == 3 or layer == 4 or layer == 5):
            best_loss = current_loss
        
        # Check if the current epoch achieved the lowest observed loss, if so, save epoch prediction
        if current_loss < best_loss:
            # Update best parameters
            best_loss = current_loss
            best_pred = epoch_pred_agg
            best_pred_dense = epoch_pred_dense
            best_actual = epoch_actual
            best_epoch = epoch
            best_layer = layer

            # Save prediction tensors
            # Training
            best_train_input = train_input
            best_train_target = train_target
            
            # Test
            best_test_input = test_input
            best_test_target = test_target
            
            # Validation
            best_vali_input = vali_input
            best_vali_target = vali_target
            
            # Detach from graph for next layer
            # Training
            best_train_input = best_train_input.detach()
            best_train_target = best_train_target.detach()
            
            # Test
            best_test_input = best_test_input.detach()
            best_test_target = best_test_target.detach()
            
            # Validation
            best_vali_input = best_vali_input.detach() 
            best_vali_target = best_vali_target.detach()
            
            # Normalise lcf
            vali_lcf, orig_vali_input = best_vali_input[:, :, -len(targets):], best_vali_input[:, :, :-7]
            train_lcf, orig_train_input = best_train_input[:, :, -len(targets):], best_train_input[:, :, :-7]
            test_lcf, orig_test_input = best_test_input[:, :, -len(targets):], best_test_input[:, :, :-7]
            train_mean = train_lcf.mean(dim=0)
            train_std = train_lcf.std(dim=0)
            train_lcf = (train_lcf - train_mean) / train_std
            test_lcf = (test_lcf - train_mean) / train_std
            vali_lcf = (vali_lcf - train_mean) / train_std
            
            # Cocatenate back
            best_train_input_norm = torch.cat([orig_train_input, train_lcf], dim=-1)
            best_test_input_norm = torch.cat([orig_test_input, test_lcf], dim=-1)
            best_vali_input_norm = torch.cat([orig_vali_input, vali_lcf], dim=-1)
            
            # Concatenate train and test
            best_traintest_input_norm = torch.cat([best_train_input_norm, best_test_input_norm], dim=0)
            best_traintest_target = torch.cat([best_train_target, best_test_target], dim=0)
            
            # Create data sets
            new_train_dataset = TensorDataset(best_traintest_input_norm, best_traintest_target)
            new_vali_dataset = TensorDataset(best_vali_input_norm, best_vali_target)
            new_input_length = best_traintest_input_norm.size(-1)
            new_input_size = best_traintest_input_norm.size(0)
    
print("\n", "Done")

#### See losses

In [ ]:
# Average loss per epoch
ltrain_epochs = []
for epoch_list in losses_train_epochs:
  epoch_avg = sum(epoch_list) / len(train_loader)
  ltrain_epochs.append(epoch_avg)

ltest_epochs = []
for epoch_list in losses_test_epochs:
  epoch_avg = sum(epoch_list) / len(test_loader)
  ltest_epochs.append(epoch_avg)

# Print epoch achieving minimum vali loss
min_value = min(ltest_epochs)
min_index = ltest_epochs.index(min_value)

print("The lowest loss:", round(min_value,3), "is found at epoch:", min_index, "\n")

# Plot the losses over time
plt.plot(ltrain_epochs, label="Training loss")
plt.plot(ltest_epochs, label="Validation loss")
plt.xlabel('Epoch')
plt.ylabel('Average loss')
plt.legend()
plt.show()

#### See predictions

In [ ]:
import math
from statistics import mean
from sklearn.metrics import mean_squared_error

# # Make lists of predictions and actual fractions (reference)
nested_pred = best_pred # results from the epoch with the minimum loss is taken
nested_actual = best_actual

unnested_pred = []
for data in nested_pred:
  for batch in data:
    for timestep in batch:
      for target in range(len(timestep)):
        unnested_pred.append(timestep[target])

unnested_actual = []
for data in nested_actual:
  for batch in data:
    for timestep in batch:
      for target in range(len(timestep)):
        unnested_actual.append(timestep[target])

# These lists contain output for entire predictions 
# Now retain lists for each class
pred_class = {}
true_class = {}

# Also retain dense predictions
pred_dense_class = {}
nested_dense_pred = best_pred_dense

unnested_dense_pred = []
for data in nested_dense_pred:
  for batch in data:
    for timestep in batch:
      for target in range(len(timestep)):
        unnested_dense_pred.append(timestep[target])

# Initialize lists for each class in predictions
for i in range(len(targets)):
  pred_class[f'{targets[i]}'] = unnested_pred[i::len(targets)]

# And dense
for i in range(len(targets)):
  pred_dense_class[f'{targets[i]}'] = unnested_dense_pred[i::len(targets)]

# Initialize lists for each class in reference data
for i in range(len(targets)):
  true_class[f'{targets[i]}'] = unnested_actual[i::len(targets)]

RMSEavg = 0
MAEavg = 0

# Plot the lists as graphs

# Loop through the data and plot the actual and predicted values
for i in range(len(targets)):
    # Get the data for the current class
    true = true_class[f'{targets[i]}']
    predicted = pred_class[f'{targets[i]}']

    # Define the x-axis data as a range of values from 0 to the length of the data
    x = range(len(true))

    # Create a new figure
    fig = plt.figure(i)

    # Create a figure with certain size
    fig = plt.figure(figsize=(20, 3))

    # Create axes
    ax = fig.add_subplot(1, 1, 1)

    # Plot the actual and predicted values
    ax.plot(x, true, label='Actual')
    ax.plot(x, predicted, label='Predicted')

    # Add a legend
    ax.legend()

    # Set the title using the class name
    var_name = f'{targets[i]}'
    ax.set_title(var_name)

    # Show the figure
    plt.show()

    # Print RMSE / MAE
    rmse = mean_squared_error(predicted, true) ** 0.5
    RMSEavg = RMSEavg + rmse
    print(f'RMSE for {var_name}: {round(rmse, 2)}')

    difference = [abs(predicted - true) for predicted, true in zip(predicted, true)]
    mae = mean(difference)
    MAEavg = MAEavg + mae
    print(f'MAE for {var_name}: {round(mae, 2)}')

print("\n")
RMSEavg = RMSEavg / len(targets)
MAEavg = MAEavg / len(targets)

print(f'Average RMSE is {round(RMSEavg, 2)} and average MAE is {round(MAEavg, 2)}')

### Create corresponding ID lists
Because of NaN values we have to recreate the ID lists

In [ ]:
# tensor_Agg_IDs and tensor_Dense_IDs
# IDs_aggregated = ['sample_id', 'location_id', 'reference_year', 'x', 'y']
# IDs_dense = ['location_id', 'date', 'x', 'y']

# Mention which IDs should be taken
predicts = "RF"

if predicts == "RF":
    tensor_Agg = tensor_Agg_IDs
    tensor_Dense = tensor_Dense_IDs
else:
    tensor_Agg = LSTM_tensor_Agg_IDs
    tensor_Dense = LSTM_tensor_Dense_IDs

Agg_dict = {}
Dense_dict = {}

for i, id_name in enumerate(IDs_aggregated):
    id_list = tensor_Agg[:, :, i].flatten().tolist()
    Agg_dict[id_name] = id_list

for i, id_name in enumerate(IDs_dense):
    id_list = tensor_Dense[:, :, i].flatten().tolist()
    Dense_dict[id_name] = id_list
   
print("Let's check if the created ID lists have the proper length:")
print(len(pred_class[f'{targets[1]}']), len(Agg_dict[f'{IDs_aggregated[1]}']))
print(len(pred_dense_class[f'{targets[1]}']), len(Dense_dict[f'{IDs_aggregated[1]}']))

### Join predictions with IDs and write to file

In [ ]:
# Add IDs (vali data was not shuffled so can add like this)
import pandas as pd

pred_df = pd.DataFrame.from_dict(Agg_dict)
dense_pred_df = pd.DataFrame.from_dict(Dense_dict)

# Adds predictions to df 
for i in range(len(targets)):
    data = pred_class[f'{targets[i]}']
    data_dense = pred_dense_class[f'{targets[i]}']

    pred_df[targets[i]] = data 
    dense_pred_df[targets[i]] = data_dense 

# Show df
pred_df.head(12)

In [ ]:
# Write both dense and dense -> aggregated predictions to file

pred_df.to_csv('Output/RF_PostLSTM_dense_to_agg.csv')
dense_pred_df.to_csv('Output/RF_PostLSTM_dense.csv')